# Weather App

In [112]:
import requests
import pandas as pd
from datetime import datetime
from geopy.geocoders import Nominatim


# Enter the city
city_name = input("Enter the city name: ")

# Get latitude and longitude of the city using Geopy with  User-Agent

def get_coordinates(city_name):

    geolocator = Nominatim(user_agent="my_weather_app")
    location = geolocator.geocode(city_name)
    if location:
        return location.latitude, location.longitude

lat, long = get_coordinates(city_name)

#print(lat, long)

# Open-Meteo API endpoint for current weather data
url = f'https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={long}&current_weather=true&timezone=auto'

# send a request to the API
response = requests.get(url)
curent_data = response.json()


# icons
temperature_icon = "🌡️"
wind_icon = "🌬️"

temp_celsuis = curent_data['current_weather']['temperature']
# weather in Fahrenheit
temp_fahrenheit = (temp_celsuis * 9/5) + 32

# wind spped
wind_speed = curent_data['current_weather']['windspeed']
# Extract the required information
weather_data = {
    'Date': datetime.now().strftime('%Y-%m-%d'),
    'City': city_name,
    'Temperature (Celsius)':f"{temp_celsuis}°C {temperature_icon}",
    'Temperature (Fahrenheit)': f"{temp_fahrenheit}°F {temperature_icon}",
    'Weather': '☀️',
    'Wind Speed': f'{wind_speed} Km/h {wind_icon}'
}

# Convert to dataFrame
df = pd.DataFrame([weather_data])

# Display
df.head()

Enter the city name: paris 


,Date,City,Temperature (Celsius),Temperature (Fahrenheit),Weather,Wind Speed
0,2024-09-02,paris,19.1°C 🌡️,66.38°F 🌡️,☀️,3.6 Km/h 🌬️


# Extra 5-days Forecast

In [113]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError

# Step 1: Choose the city and get its coordinates
city_name = input("Enter the city name: ")

# city latitude and longitude using geopy with User-Agent
def get_coordinates(city_name):

    geolocator = Nominatim(user_agent="my_weather_app")
    location = geolocator.geocode(city_name)
    if location:
        return location.latitude, location.longitude

lat, lon = get_coordinates(city_name)
#lat, lon

Enter the city name: paris


In [114]:
# Calculate the date range for the last year
end_date = datetime.now()
start_date = end_date - timedelta(days=365)

# Format the dates as required by the API
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

# Open-Meteo API endpoint for historical weather data
url = f'https://archive-api.open-meteo.com/v1/archive?latitude={lat}&longitude={lon}&start_date={start_date_str}&end_date={end_date_str}&daily=temperature_2m_max&timezone=auto'

# Make a request to the API
response = requests.get(url)
data = response.json()

# Extract the dates and temperatures from the API response
dates = data['daily']['time']
temperatures = data['daily']['temperature_2m_max']

# Create a DataFrame with the extracted data
weather_data = {
    'Date': dates,
    'City': city_name,
    'Temperature (Celsius)': temperatures
}

df = pd.DataFrame(weather_data)
# dropping missing value from the row
df = df.dropna(axis=0)

#df.tail()

In [115]:
# preprocess the data and use as index
df.set_index('Date', inplace=True)

# Ensure 'Date' index is in datetime format
df.index = pd.to_datetime(df.index)

# scaling the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[['Temperature (Celsius)']])

# Creating time series data for LSTM
# use last 10 days to predict the next day
n_input = 10
n_features = 1  # We are only using temperature
generator = TimeseriesGenerator(scaled_data, scaled_data, length=n_input, batch_size=1)

# Making LSTM model with bidirectional layers
model = Sequential([
    Bidirectional(LSTM(100, activation='relu'), input_shape=(n_input, n_features)),
    Dropout(0.2),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

# training the model
model.fit(generator, epochs=50)

# predict the next 5 days
predictions = []
current_batch = scaled_data[-n_input:].reshape((1, n_input, n_features))

# predicting for 5 days
for i in range(5):
    current_pred = model.predict(current_batch)[0]
    predictions.append(current_pred)
    current_batch = np.append(current_batch[:, 1:, :], [[current_pred]], axis=1)





Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


 16/355 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.1429

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


355/355 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - loss: 0.0425
Epoch 2/50
355/355 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.0115
Epoch 3/50
355/355 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 0.0090
Epoch 4/50
355/355 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0111
Epoch 5/50
355/355 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - loss: 0.0093
Epoch 6/50
355/355 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - loss: 0.0079
Epoch 7/50
355/355 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - loss: 0.0074
Epoch 8/50
355/355 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - loss: 0.0083
Epoch 9/50
355/355 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - loss: 0.0088
Epoch 10/50
355/355 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - loss: 0.0086
Epoch 11/50
355/355 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - loss: 0.0069
Epoch 12/50
355/355 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0081
Epoch 13/50
355/355 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - loss: 0.0070
Epoch 14/50
355/355 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 0.0075
Epoch 15/50
355/355 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step 

In [116]:
# Inverse transform the predictions
predictions = scaler.inverse_transform(predictions)
# Display the predictions and use the last sequence from the test data to start forecasting

future_dates = [df.index[-1] + timedelta(days=i) for i in range(1, 6)]
forecast_5days = pd.DataFrame(data={'Date': future_dates,'City': city_name, 'Predicted Temperature': predictions.flatten()})
print(forecast_5days)

        Date   City  Predicted Temperature
0 2024-09-02  paris              20.611625
1 2024-09-03  paris              21.140449
2 2024-09-04  paris              21.107920
3 2024-09-05  paris              21.403058
4 2024-09-06  paris              21.707071
